In [51]:
from bird_nest_project.collected_data.models import PlottedData
import pandas as pd
import folium
from geopy.distance import geodesic
from geopy.distance import distance as geopy_distance
import numpy as np

In [23]:
plotted_data_queryset = PlottedData.objects.all()
# plotted_data_queryset = PlottedData.objects.order_by('group_name').distinct('group_name')
# plotted_data_queryset = set(PlottedData.objects.values_list('group_name'))
# plotted_data_queryset = set(PlottedData.objects.values_list('group_name', flat=True))


In [24]:
print(plotted_data_queryset)

<QuerySet [<PlottedData: PlottedData object (1006)>, <PlottedData: PlottedData object (1007)>, <PlottedData: PlottedData object (1008)>, <PlottedData: PlottedData object (1009)>, <PlottedData: PlottedData object (1010)>, <PlottedData: PlottedData object (1011)>, <PlottedData: PlottedData object (1012)>, <PlottedData: PlottedData object (1013)>, <PlottedData: PlottedData object (1014)>, <PlottedData: PlottedData object (1015)>, <PlottedData: PlottedData object (1016)>, <PlottedData: PlottedData object (1017)>, <PlottedData: PlottedData object (1018)>, <PlottedData: PlottedData object (1019)>, <PlottedData: PlottedData object (1020)>, <PlottedData: PlottedData object (1021)>, <PlottedData: PlottedData object (1022)>, <PlottedData: PlottedData object (1023)>, <PlottedData: PlottedData object (1024)>, <PlottedData: PlottedData object (1025)>, '...(remaining elements truncated)...']>


In [34]:
# Convert the queryset to a list of dictionaries
data = list(plotted_data_queryset.values())

df = pd.DataFrame(data)
df_unique = df.drop_duplicates(subset='group_name')


In [35]:
df_unique

,id,group_name,file_path,format,latitude,longitude,enriched_data
0,1006,0095,source_data/0095/PXL_20240429_170324019.jpg,MPO,52.375747,4.841281,"{'id': '0095', 'nest_status': 'Built', 'active..."
5,1011,0061,source_data/0061/PXL_20240413_111811906.jpg,MPO,52.344781,4.881094,"{'id': '0061', 'nest_status': 'Built', 'active..."
8,1014,0059,source_data/0059/PXL_20240413_104111857.jpg,MPO,52.342817,4.877942,"{'id': '0059', 'nest_status': 'Built', 'active..."
11,1017,0066,source_data/0066/PXL_20240413_124231093.jpg,MPO,52.348067,4.890311,"{'id': '0066', 'nest_status': 'Built', 'active..."
18,1024,0092,source_data/0092/PXL_20240426_175718249.jpg,MPO,52.378944,4.880106,"{'id': '0092', 'nest_status': 'Built', 'active..."
...,...,...,...,...,...,...,...
584,1590,0156,source_data/0156/PXL_20240505_135137612.jpg,MPO,52.364175,4.890961,"{'id': '0156', 'nest_status': 'Built', 'active..."
588,1594,0102,source_data/0102/PXL_20240504_073341042.jpg,MPO,52.368758,4.876200,"{'id': '0102', 'nest_status': 'Built', 'active..."
590,1596,0105,source_data/0105/PXL_20240504_074242023.MP.jpg,MPO,52.363469,4.880858,"{'id': '0105', 'nest_status': 'Built', 'active..."
592,1598,0133,source_data/0133/PXL_20240505_122826711.jpg,MPO,52.347867,4.859056,"{'id': '0133', 'nest_status': 'Built', 'active..."


In [36]:
# Create a map centered at a specific location, you can adjust the location and zoom level as needed
m = folium.Map(location=[df_unique['latitude'].mean(), df_unique['longitude'].mean()], zoom_start=13.5)

In [37]:
# Define marker colors for different plastic statuses
status_colors = {
    'Not visually present': 'green',
    'Present': 'orange',
    'Prevalent': 'red'
}

In [38]:

# Add markers for each data point
for index, row in df_unique.iterrows():
    if not pd.isna(row['latitude']) and not pd.isna(row['longitude']):
        plastic_status = row['enriched_data'].get('plastic_status', 'Unknown')
        # Use the status_colors dictionary to determine marker color based on Plastic Status
        marker_color = status_colors.get(plastic_status, 'blue')
        folium.Marker([row['latitude'], row['longitude']], icon=folium.Icon(color=marker_color)).add_to(m)

# Display the map
m

In [70]:
df_unique.count()
df_valid = df_unique.dropna(subset=['latitude', 'longitude'])
df_valid.count()

id               156
group_name       156
file_path        156
format           156
latitude         156
longitude        156
enriched_data    156
dtype: int64

In [72]:
m = folium.Map(location=[df_unique['latitude'].mean(), df_unique['longitude'].mean()], zoom_start=13.5)
nests_to_check = []
# Iterate over each record and compare its distance to all other records
for index, row in df_valid.iterrows():
    point1 = (row['latitude'], row['longitude'])
    
    # Flag to determine if this point is close to any other point
    close_to_other_point = False
    
    # Iterate over all other records
    for idx, other_row in df_valid.iterrows():
        if index != idx:  # Exclude comparing the point to itself
            point2 = (other_row['latitude'], other_row['longitude'])
            
            # Calculate the distance between the two points using geodesic distance
            distance = geodesic(point1, point2).meters  # distance in meters
 
            # Check if the distance is within the threshold (e.g., 5 meters)
            if distance <= 0.1:
                close_to_other_point = True
                break  # No need to check further if one close point is found
    
    # If the point is close to any other point, plot it
    if close_to_other_point:
        nests_to_check.append(row)
        plastic_status = row['enriched_data'].get('plastic_status', 'Unknown')
        marker_color = status_colors.get(plastic_status, 'blue')
        folium.Marker(
            [row['latitude'], row['longitude']], 
            icon=folium.Icon(color=marker_color)
        ).add_to(m)

# Display the map
m

In [76]:
df_nests_to_check = pd.DataFrame(nests_to_check, columns=['group_name', 'latitude', 'longitude'])

# Export the DataFrame to a CSV file
df_nests_to_check.to_csv('corrections.csv', index=False)

In [ ]:
nest_ids_to_check = [
    "0005", "0007", "0010", "0011", "0012", "0027", "0028", "0031", "0032", "0033", 
    "0068", "0069", "0070", "0071", "0072", "008", "0101", "0102", "0104", "0105", 
    "0106", "0107", "0108", "0109", "0110", "0114", "0115", "0116", "0119", "0120", 
    "0123", "0124", "0128", "0129", "0130", "0133", "0140", "0141", "0142", "0143", 
    "0144", "0147", "0152", "0153", "0154", "0155"
]